In [3]:
## https://www.r-bloggers.com/2015/12/tutorial-rna-seq-differential-expression-pathway-analysis-with-sailfish-deseq2-gage-and-pathview/
library("AnnotationDbi")
library("org.Hs.eg.db")
library("enrichplot")
library("clusterProfiler")

> ## VDAART

In [4]:
## Load DE genes from VDAART
res <- read.csv('Supplementary/Tab_S2.csv', row.names = 1 ) 

> Only works with R-KEGG

In [5]:
de_genes_list <-  res 
de_genes_list  <- de_genes_list

# Create a vector of the gene unuiverse
gene_list <- de_genes_list$logFC

# Name vector with ENTREZ ids
names(gene_list) <- de_genes_list$entrezgene_id 

# omit any NA values 
gene_list <- na.omit(gene_list)

# sort the list in decreasing order (required for clusterProfiler)
gene_list = sort(gene_list, decreasing = TRUE)

go <- enrichKEGG(gene         = names(gene_list),
                organism     = 'hsa',
                pvalueCutoff = 0.05,
                # nPerm        = 10000,
                minGSSize    = 10,
                maxGSSize    = 2000,
                pAdjustMethod = "BH")

go <- as.data.frame(setReadable(go, 'org.Hs.eg.db',keyType="ENTREZID"))
li_gr = c()
li_br = c()
for (x in go$GeneRatio) {
  li_gr <- append(li_gr, (eval(parse(text=x))) )
}

for (x in go$BgRatio) {
  li_br <- append(li_br, (eval(parse(text=x))) )
}
tmp <- data.frame(go)

tmp$FE <- li_gr/li_br ## Fold enrichment
tmp$log10adjpval <- -log10(tmp$p.adjust)
tmp$rankFDR <- c(seq(1, nrow(tmp)))
tmp <- tmp[order( -tmp$FE),]
tmp$rankFE <- seq(1, nrow(tmp))
tmp$rank <- (tmp$rankFDR + tmp$rankFE)/2

tab <- tmp[order(tmp['log10adjpval']),]

tab <- tab[c('ID','Description','FE','log10adjpval','Count', 'geneID', 'pvalue', 'p.adjust')]
tab$nGenes <- tab$Count
tab['-Log10FDR'] <- tab$log10adjpval
tab <- tab[order(tab$FE),]
tab$Pathway <- factor(tab$Description, levels = tab$Description)
dim(tab)
tab


Reading KEGG annotation online: "https://rest.kegg.jp/link/hsa/pathway"...

Reading KEGG annotation online: "https://rest.kegg.jp/list/pathway/hsa"...

Warning message in xtfrm.data.frame(x):
“cannot xtfrm data frames”


[1] 11 11

,ID,Description,FE,log10adjpval,Count,geneID,pvalue,p.adjust,nGenes,-Log10FDR,Pathway
,<chr>,<chr>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<fct>
hsa05202,hsa05202,Transcriptional misregulation in cancer,2.286921,1.340618,16,PBX1/ITGAM/SPI1/GADD45B/BCL6/BCL2A1/CEBPA/CD86/FLT3/CEBPB/CEBPE/PTCRA/FCGR1A/MMP9/CD14/IL1R2,1.701974e-03,4.564385e-02,16,1.340618,Transcriptional misregulation in cancer
hsa04613,hsa04613,Neutrophil extracellular trap formation,2.455297,1.564758,17,ITGAM/MAPK14/NCF1/FCGR2A/NCF2/NCF4/TLR2/PADI4/FPR2/CAMP/AZU1/TLR8/AQP9/FPR1/TLR4/FCGR3B/FCGR1A,5.541448e-04,2.724221e-02,17,1.564758,Neutrophil extracellular trap formation
hsa04145,hsa04145,Phagosome,2.540815,1.402150,14,ITGAM/NCF1/FCGR2A/THBS1/CTSS/OLR1/NCF2/NCF4/TLR2/FCAR/TLR4/FCGR3B/FCGR1A/CD14,1.208568e-03,3.961416e-02,14,1.402150,Phagosome
hsa05152,hsa05152,Tuberculosis,2.758599,2.060333,18,ITGAM/TLR1/VDR/NOD2/MAPK14/IFNGR2/IFNGR1/FCGR2A/CTSS/CEBPB/FCER1G/TLR2/CAMP/CLEC4E/TLR4/FCGR3B/FCGR1A/CD14,8.850476e-05,8.702968e-03,18,2.060333,Tuberculosis
hsa04668,hsa04668,TNF signaling pathway,2.903788,1.510212,12,PTGS2/FOS/CREB5/NOD2/JUNB/MAPK14/BCL3/CEBPB/JAG1/IL18R1/SOCS3/MMP9,8.376367e-04,3.088785e-02,12,1.510212,TNF signaling pathway
hsa04625,hsa04625,C-type lectin receptor signaling pathway,3.182999,1.670307,12,PTGS2/PRKCD/MAPK14/RRAS/PYCARD/BCL3/NLRP3/FCER1G/CLEC4E/CLEC1B/CLEC4D/CLEC6A,3.621107e-04,2.136453e-02,12,1.670307,C-type lectin receptor signaling pathway
hsa00010,hsa00010,Glycolysis / Gluconeogenesis,3.705580,1.564758,9,BPGM/PGAM1/HK2/LDHA/GAPDH/FBP1/ALDH3B1/HK3/ALDH2,6.464253e-04,2.724221e-02,9,1.564758,Glycolysis / Gluconeogenesis
hsa05134,hsa05134,Legionellosis,4.433462,1.921541,9,ITGAM/HSPA1A/PYCARD/TLR5/TLR2/NAIP/TLR4/NLRC4/CD14,1.624413e-04,1.198005e-02,9,1.921541,Legionellosis
hsa04380,hsa04380,Osteoclast differentiation,4.495494,6.126594,22,FOS/SPI1/GAB2/JUNB/MAPK14/NCF1/IFNGR2/FOSL2/IFNGR1/FCGR2A/SIRPB1/LILRA2/LILRB3/NCF2/NCF4/OSCAR/TYROBP/SOCS3/FCGR3B/FCGR1A/LILRB4/LILRA5,2.532704e-09,7.471477e-07,22,6.126594,Osteoclast differentiation


In [6]:
library(ggplot2)
pdf("Supplementary/Fig_S2_Kegg.pdf", width = 7, height = 4)
size = 12

# Horizontal version
p <- ggplot(data = tab, aes(x = Pathway, y = FE ) ) + 
# scale_color_gradient(low = "#a6b4a6", high = "#404e40") +
 scale_color_gradient(low = "#ffa0b1", #"#ff617d", 
                      high = "#b24357") +
  geom_segment(  aes(x=Pathway, xend=Pathway, y=0, yend=FE, color = `-Log10FDR`, size = 1), show.legend = F ) +
  geom_point(data = tab, aes(color = `-Log10FDR`, size = nGenes)) + 
  theme_light() +
  coord_flip() +
  xlab("PATHWAYS") + 
  ylab("Fold Enrichment") + 
  ggtitle("KEGG ANALYSIS")+
  theme(
    panel.grid.major.y = element_blank(),
    axis.ticks.y = element_blank(),
    axis.text.x = element_text(size=15, angle=0),
    axis.text.y = element_text(size=15, angle=0),
    plot.title = element_text(size=12, face='bold', hjust = 0.5),
    panel.background = element_blank(),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    panel.border = element_rect(colour = "grey", fill=NA, size=1),
    axis.title.y = element_text(size = size),
    axis.title.x = element_text(size = size), 
      legend.title = element_text(size=12),
      legend.text = element_text(size=12)
  )
p
dev.off()
  

Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”
Warning message:
“The `size` argument of `element_rect()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”


png 
  2